In [31]:
import sys
import pathlib as pl
home = pl.Path.home()
omph_path = home / 'work' / 'Omphalos'
tope_path = home / 'work' / 'topepan'

sys.path.append(str(omph_path))
sys.path.append(str(tope_path))

import omphalos
import topepan as tp
from topepan import plot as tpp
from omphalos import file_methods as fm
from coeus import helper as hp
import xarray as xr

In [32]:
# Read in data for dict.
pickle = 'cr_08-03-23.pkl'
path = home / 'work' / 'data' / 'hydrothermal_alteration' / pickle
raw = fm.unpickle(path)

set1, errors1 = hp.filter_errors(raw)
set1 = hp.fix_smalls(set1, 'volume')

#raw = fm.unpickle('/home/angus/work/data/hydrothermal_alteration/700_2deg.pkl')

#set2, errors2 = hp.filter_errors(raw)
#set2 = hp.fix_smalls(set2, 'volume')

datasets = [set1]

Returned 29 files without errors out of a total possible 29.
0 files had errors.
0 files had unhandled errors.
File failure rate: 0.0 %.
To see unhandled errors, run with verbose=True.


In [33]:
for i in raw:
    print(set1[i].results['volume'].time[-1].to_numpy())

11000.0
9000.0
7000.0
6000.0
6000.0
5000.0
5000.0
6000.0
6000.0
5000.0
2200.0
5000.0
5000.0
5000.0
6000.0
6000.0
5000.0
6000.0
6000.0
6000.0
0.1
0.1
0.1
6000.0
6000.0
6000.0
6000.0
6000.0
2000.0


In [39]:
def calc_clays(file):
    clays = ['Hsaponite(Mg)', 'Clinochlore', 'Chamosite(Daph', 'Celadonite']
    clays_init = xr.zeros_like(file.results['volume']['Clinochlore'])
    file.results['volume'] = file.results['volume'].assign({'clays': clays_init})

    for clay in clays:
        file.results['volume']['clays'] += file.results['volume'][clay]

for file in set1:
    calc_clays(set1[file])

In [37]:
set1[0].results['volume'].isel(time=20)


<xarray.Dataset>
Dimensions:         (X: 500, Y: 1, Z: 1)
Coordinates:
  * X               (X) float64 1.0 3.0 5.0 7.0 9.0 ... 993.0 995.0 997.0 999.0
  * Y               (Y) float64 0.5
  * Z               (Z) float64 0.5
    time            float64 6e+03
Data variables: (12/24)
    Forsterite      (X, Y, Z) float64 0.03 0.03 0.03 ... 2.704e-34 5.3e-35
    Fayalite        (X, Y, Z) float64 0.03 0.03 0.03 ... 9.009e-49 8.347e-50
    Microcline      (X, Y, Z) float64 0.01 0.01 0.01 ... 1.112e-05 8.688e-06
    Ferrosilite(al  (X, Y, Z) float64 0.05 0.05 0.05 ... 0.02722 0.02682 0.02642
    Enstatite(alph  (X, Y, Z) float64 0.05 0.05 0.05 ... 0.03204 0.03171 0.03137
    Diopside        (X, Y, Z) float64 0.19 0.19 0.19 ... 0.0833 0.08261 0.08207
    ...              ...
    Clinozoisite    (X, Y, Z) float64 1e-10 1e-10 1e-10 ... 0.09637 0.09604
    Chamosite(Daph  (X, Y, Z) float64 1e-10 1e-10 1e-10 ... 2.666e-23 7.302e-24
    Clinochlore     (X, Y, Z) float64 1.001e-10 1.003e-10 ... 0.03353 0.03365
    Natrolite       (X, Y, Z) float64 1e-10 1e-10 1e-10 ... 3.137e-11 2.491e-11
    Anhydrite       (X, Y, Z) float64 8.702e-11 8.675e-11 ... 1.732e-44
    Calcite         (X, Y, Z) float64 1.016e-10 1.017e-10 1.018e-10 ... 0.0 0.0

In [40]:
%matplotlib notebook

import matplotlib as mpl
from matplotlib import pyplot as plt
from cmcrameri import cm
import numpy as np
mpl.rcParams.update(mpl.rcParamsDefault)

mpl.rcParams['axes.linewidth'] = 3
mpl.style.use('seaborn-poster')

time_steps = 25
i = 0
file_num=0
fontsize = 24

fig, axes = plt.subplots(1,4, figsize=(20,11))
plot_vars = ['clays'] * 4

time = 20

#Cell vol /cm^3
cell_vol = 200 * 100**2
#Calcite molar volume cm^3/mol
calcite_mv = 36.934
#Calcite molar mass g/mol
cal_mm = 100.087
# Calcite density
c_den = cal_mm / calcite_mv
#vf_to_mass = cell_vol / calcite_mv * cal_mm
vf_to_mass = cell_vol * c_den / 1000
vf_to_moles = cell_vol / calcite_mv
scale_factors = (vf_to_mass, vf_to_mass, vf_to_mass, vf_to_mass)
scale_factors = (vf_to_moles, vf_to_moles, vf_to_moles, vf_to_moles)
scale_factors = (1,1,1,1)

#range_sets = [(1,6), (7,10), (11,15), (16,20)]
# Temporarily change range set to account for missing file at start.
range_sets = [(1,6), (7,10), (11,15), (16,20)]
labels = [[10, 20, 30, 40, 50, 60],
          [10, 20, 30, 40],
          [1, 10, 20, 30, 40],
          [2.0, 2.2, 2.4, 2.6, 2.8]]

linestyles = ['-', '--']
legend_titles = ['[Mg$\mathbf{^{2+}}$] / mM', '[Ca$\mathbf{^{2+}}$] / mM', '[SO$\mathbf{_4}}$] / mM', '[HCO$\mathbf{_3}^-$] / mM']

x_lims= [(0,10), (0,6), (0,6), (0,6)]
cols = ['r', 'b', 'g', 'm', 'k', 'c']
temperatures = [15, 2]

iterables = zip(plot_vars, axes, legend_titles, scale_factors, range_sets, labels)

save_points = [0, 1, 2, 3, 5, 7, 9, 11, 19, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]

j=0

#Prep fig before plotting
for axis, x_lim in zip(axes, x_lims):
    axis.xaxis.tick_top()
    axis.xaxis.set_label_position('top')
    axis.grid(False)
    axis.tick_params(length=8, width=4)
    axis.set_xlabel('Clays / volume fraction', fontweight='bold')
    axis.set_ylabel('Depth / m', fontweight='bold')
    #axis.set_ylim(0,200)
    #axis.set_xlim(0,45)
    for tick in axis.xaxis.get_major_ticks():
        tick.label2.set_fontweight('bold')
    for tick in axis.yaxis.get_major_ticks():
        tick.label1.set_fontweight('bold')
    axis.invert_yaxis()

for dataset, ls, temperature in zip(datasets, linestyles, temperatures):

    iterables = zip(plot_vars, axes, legend_titles, scale_factors, range_sets, labels)
    # Axis level loop
    for plot_var, axis, legend_title, scale, range_set, label_a in iterables:
        # Line set loop (i.e. groups of lines sharing a property)
        # Line instance loop
        index = np.arange(range_set[0]-1, range_set[1])
        for ind, label, c in zip(index, label_a, cols):
            if i == ind:
               axis.set_prop_cycle(None)
            try:
                x = dataset[ind].results['volume'].isel(time=time, Y=0, Z=0)[plot_var].values * scale
                y = dataset[ind].results['volume'].isel(time=time, Y=0, Z=0)['X'].values
            except IndexError:
                x=0
                y=0
            axis.plot(x, y, label=f'{label}, T={temperature}C', linestyle=ls, c=c)
            axis.legend(title=legend_title, prop={'weight': 'bold', 'size': 15}, title_fontproperties={'weight': 'bold', 'size': 15}, framealpha=0, loc=4)

fig.tight_layout()

fig.savefig('calcite.png', dpi=150)
fig.show()

/var/folders/86/d5n0l2wj5yn0p84q_4jzm_6r0000gn/T/ipykernel_12102/2451998863.py:10: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mpl.style.use('seaborn-poster')


<IPython.core.display.Javascript object>